In [1]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import os
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Set API keys
os.environ["OPENAI_API_KEY"] = "..."
os.environ["WOLFRAM_ALPHA_APPID"] = ""

In [4]:
def mock_retriever(query):
    mock_knowledge_base = {
        "Геометрія: трикутники": "Трикутник - це багатокутник з трьома сторонами. Формула площі: S = 0.5 * основа * висота.",
        "Алгебра: рівняння": "Рівняння - це математичне твердження, що два вирази рівні. Наприклад, 2x + 5 = 15.",
        "Дроби: ділення": "Ділення дробів: помножте перший дріб на обернений другий дріб."
    }
    return mock_knowledge_base.get(query, "На жаль, я не можу знайти інформацію про цю тему.")

In [14]:
from langchain.prompts import PromptTemplate

task_prompt = PromptTemplate(
    input_variables=["topic", "complexity", "retrieved_context"],
    template="""
    Використовуючи таку інформацію:
    {retrieved_context}

    Згенеруй математичне завдання на тему "{topic}" з рівнем складності "{complexity}".
    Формат: Завдання: ..., Відповідь: ...
    """
)

In [24]:
topic = "Геометрія: Циліндри"
complexity = "середній"
task_type = "problem"
retrieved_context = mock_retriever(topic)

# Generate task
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [25]:
task_query = task_prompt.format(
    topic=topic,
    complexity=complexity,
    type=task_type,
    retrieved_context=retrieved_context
)
print("Generated Query for Task:")
print(task_query)

Generated Query for Task:

    Використовуючи таку інформацію:
    На жаль, я не можу знайти інформацію про цю тему.

    Згенеруй математичне завдання на тему "Геометрія: Циліндри" з рівнем складності "середній".
    Формат: Завдання: ..., Відповідь: ...
    


In [26]:
from langchain.chains import LLMChain

# Define LLMChain with the prompt
task_chain = LLMChain(llm=llm, prompt=task_prompt)

# Run the chain to generate the task
task = task_chain.run({
    "topic": topic,
    "complexity": complexity,
    "type": task_type,
    "retrieved_context": retrieved_context
})
print("Generated Task and Answer:")
print(task)

Generated Task and Answer:
Завдання: Обчисліть об'єм циліндра, якщо його радіус дорівнює 5 см, а висота - 10 см. (Відомо, що об'єм циліндра обчислюється за формулою V = πr^2h, де r - радіус циліндра, h - висота циліндра)
Відповідь: 785.4 см^3


In [28]:
task_text = task.split("Завдання:")[1].split("Відповідь:")[0].strip()

task_text

"Обчисліть об'єм циліндра, якщо його радіус дорівнює 5 см, а висота - 10 см. (Відомо, що об'єм циліндра обчислюється за формулою V = πr^2h, де r - радіус циліндра, h - висота циліндра)"

In [31]:
wolfram = WolframAlphaAPIWrapper()

In [37]:
task_text = 'Calculate the volume of a cylinder if its radius is 5 cm and its height is 10 cm.'

In [38]:
wolfram_answer = wolfram.run(task_text)
print("Validated Correct Answer (Wolfram Alpha):")
print(wolfram_answer)

ReadTimeout: 

In [17]:
wolfram = WolframAlphaAPIWrapper()

In [18]:
wolfram.run("What is 2x+5 = -3x + 7?")

'Assumption: 2 x + 5 = -3 x + 7 \nAnswer: x = 2/5'

In [19]:
# Tools for LangChain agent
tools = [
    Tool(
        name="Wolfram Alpha",
        func=wolfram.run,
        description="Use this for mathematical calculations and validations."
    ),
    Tool(
        name="Mock Knowledge Retriever",
        func=retrieve_mocked_knowledge,
        description="Mock retriever to simulate knowledge base for math topics."
    )
]


In [20]:
# Initialize the LangChain agent
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

/tmp/ipykernel_223418/949775598.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
/tmp/ipykernel_223418/949775598.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's 

In [21]:
# Custom task generation and solving prompt
task_prompt = PromptTemplate(
    input_variables=["topic", "retrieved_context"],
    template="""
    Використовуючи таку інформацію:
    {retrieved_context}

    Згенеруй математичне завдання на тему "{topic}" українською мовою.
    Додай правильну відповідь після завдання.
    Формат: Завдання: ..., Відповідь: ...
    """
)

In [24]:
# Task Generation Function
def generate_math_task(topic):
    # Retrieve mock context
    retrieved_context = retrieve_mocked_knowledge(topic)
    if retrieved_context is None:
        return "Unable to retrieve context for the given topic."
    # Create task with context
    task = task_prompt.format(topic=topic, retrieved_context=retrieved_context)
    return agent.run(task)

In [25]:
# Example Usage
topic = "Геометрія: трикутники"
result = generate_math_task(topic)
print("Generated Task and Solution:")
print(result)



> Entering new AgentExecutor chain...
I need to generate a math problem related to triangles in Ukrainian.
Action: Mock Knowledge Retriever
Action Input: Generate math problem related to triangles in Ukrainian
Observation: На жаль, я не можу знайти інформацію про цю тему.
Thought:I will try using my own knowledge to create a math problem related to triangles in Ukrainian.
Final Answer: Завдання: Обчисліть площу трикутника, якщо його основа дорівнює 6, а висота 8. Відповідь: 24.

> Finished chain.
Generated Task and Solution:
Завдання: Обчисліть площу трикутника, якщо його основа дорівнює 6, а висота 8. Відповідь: 24.
